In [ ]:
import pandas as pd
import numpy as np
import pyodbc
import datetime
from tqdm import tqdm
import math

window=7

server = ''
database = ''
username = ''
password = ''
driver= 'SQL Server'
connection = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
requestString_sessions = '''
...
'''
df = pd.read_sql_query(requestString_sessions,connection)


def detect_cusum(x, threshold=1, drift=0, ending=False):
    x = np.atleast_1d(x).astype('float64')
    gp, gn = np.zeros(x.size), np.zeros(x.size)
    ta, tai, taf = np.array([[], [], []], dtype=int)
    tap, tan = 0, 0
    amp = np.array([])
    # Find changes (online form)
    for i in range(1, x.size):
        s = x[i] - x[i-1]
        gp[i] = gp[i-1] + s - drift  # cumulative sum for + change
        gn[i] = gn[i-1] - s - drift  # cumulative sum for - change
        if gp[i] < 0:
            gp[i], tap = 0, i
        if gn[i] < 0:
            gn[i], tan = 0, i
        if gp[i] > threshold or gn[i] > threshold:  # change detected!
            ta = np.append(ta, i)    # alarm index
            tai = np.append(tai, tap if gp[i] > threshold else tan)  # start
            gp[i], gn[i] = 0, 0      # reset alarm
    # THE CLASSICAL CUSUM ALGORITHM ENDS HERE

    # Estimation of when the change ends (offline form)
    if tai.size and ending:
        _, tai2, _, _ = detect_cusum(x[::-1], threshold, drift)
        taf = x.size - tai2[::-1] - 1
        # Eliminate repeated changes, changes that have the same beginning
        tai, ind = np.unique(tai, return_index=True)
        ta = ta[ind]
        # taf = np.unique(taf, return_index=False)  # corect later
        if tai.size != taf.size:
            if tai.size < taf.size:
                taf = taf[[np.argmax(taf >= i) for i in ta]]
            else:
                ind = [np.argmax(i >= ta[::-1])-1 for i in taf]
                ta = ta[ind]
                tai = tai[ind]
        # Delete intercalated changes (the ending of the change is after
        # the beginning of the next change)
        ind = taf[:-1] - tai[1:] > 0
        if ind.any():
            ta = ta[~np.append(False, ind)]
            tai = tai[~np.append(False, ind)]
            taf = taf[~np.append(ind, False)]
        # Amplitude of changes
        amp = x[taf] - x[tai]

    return ta, tai, taf, amp

ta, tai, taf, amp = detect_cusum(df['status_reply_p'], 0.02, 0, True)

points = np.concatenate((ta, tai, taf), axis=0)


df['anomaly']=0
df['anomaly_delta']=0
df['anomaly_degree']=0


for i in range(len(df)):
    if i in points:
        df['anomaly'].iloc[i]=1

for i in range(len(df)):
    j=i-window
    if j<0:
        df['anomaly_delta'].iloc[i] = 0
    else:
        prev_mean = np.mean(df['status_reply_p'].iloc[j:i])
        curr_mean = df['status_reply_p'].iloc[i]
        df['anomaly_delta'].iloc[i]= - prev_mean + curr_mean

for i in range(len(df)):
    if df['anomaly'].iloc[i]==1:
        if df['anomaly_delta'].iloc[i]<=-0.01:
            df['anomaly_degree'].iloc[i]=-2
        elif df['anomaly_delta'].iloc[i]<0:
            df['anomaly_degree'].iloc[i]=-1
        elif df['anomaly_delta'].iloc[i]==0:
            df['anomaly_degree'].iloc[i]=0
        elif df['anomaly_delta'].iloc[i]<=0.01:
            df['anomaly_degree'].iloc[i]=1
        else:
            df['anomaly_degree'].iloc[i]=2
    else:
        df['anomaly_degree'].iloc[i]=0


cursor = connection.cursor()
cursor.execute("TRUNCATE TABLE [ChatBotData].[report].[anomaly_day]")
connection.commit()
cursor.close()

cursor = connection.cursor()
for index, row in df.iterrows():
    cursor.execute("INSERT INTO [ChatBotData].[report].[anomaly_day] (date,total,status_reply,status_reply_p,status_transfer,status_transfer_p,status_transfer_reply,status_transfer_reply_p,anomaly,anomaly_delta,anomaly_degree) values(?,?,?,?,?,?,?,?,?,?,?)",
                  row.date,row.total,row.status_reply,row.status_reply_p,row.status_transfer,row.status_transfer_p,row.status_transfer_reply,row.status_transfer_reply_p,row.anomaly,row.anomaly_delta,row.anomaly_degree)
connection.commit()
cursor.close()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
